# Huffman Coding in C++

* 結果是對的但有 memory leaking，只有 ```new``` 沒有 ```delete```，下面的 dtor 寫的不對
* coding 結果和 python 不一樣，可能是因為 priority queue 處理同 freq 的 node 方法不同

In [6]:
#include <iostream>
#include <string>
#include <queue>       // std::priority_queue
#include <vector>
#include <utility>     // std::pair, std::swap
#include <functional>  // std::greater
#include <unordered_map>

class HeapNode
{
public: 
    HeapNode* left = nullptr;
    HeapNode* right = nullptr;
    int freq;
    char ch;
    
    HeapNode(int freq, char ch, HeapNode* left, HeapNode* right) 
        : freq(freq), ch(ch), left(left), right(right) {}
    
    HeapNode(const HeapNode& other) { 
        HeapNode* p = clone(&other); 
        
        this->freq = p->freq;
        this->ch = p->ch;
        this->left = p->left;
        this->right = p->right;
    }
 
//     ~HeapNode() { 
//         deleteNode(this->left);
//         deleteNode(this->right);
//     }
 
//     void deleteNode(const HeapNode* p) {
//         if (p){
//             deleteNode(p->left);
//             deleteNode(p->right);
//             delete p;
//         }
//     }
    
    HeapNode* clone(const HeapNode* other) {
        HeapNode* p = new HeapNode(other->freq, other->ch, nullptr, nullptr);
        
        if (other->left)
            p->left = clone(other->left);
        if (other->right)
            p->right = clone(other->right);
        
        return p;
    }

    void print() const { 
        std::cout << "(" << ch << ", " << freq << ")" << std::endl; 
    }
    
    friend bool operator>(const HeapNode& a, const HeapNode& b) { 
        return a.freq > b.freq; 
    }
};

In [7]:
class HuffmanCoding{

    std::string original_message;
    std::vector<std::pair<std::string, std::string>> codeCharPairs;
    const HeapNode* huffmanTree;
        
    const HeapNode* buildHuffmanTree()
    {
        std::unordered_map<char, int> freq;
        for (const char& c : original_message)
            freq[c]++;

        std::vector<HeapNode> nodes;
        for (const auto& pair : freq){
            char letter = pair.first;
            int frequency = pair.second;
            nodes.push_back(HeapNode(frequency, letter, nullptr, nullptr));
        }

        std::priority_queue<HeapNode, std::vector<HeapNode>, std::greater<HeapNode>> q(std::greater<HeapNode>(), nodes);

        while(q.size() > 1) 
        {
            HeapNode leftNode = q.top();  
            HeapNode* left = &leftNode;
            q.pop();

            HeapNode rightNode = q.top();
            HeapNode* right = &rightNode;
            q.pop();

            q.push(HeapNode(left->freq + right->freq, '\0', left, right));
        }
        return &(q.top());
    }
    
    std::vector<std::pair<std::string, std::string>> getCodeCharPairs(const HeapNode* p)
    {        
        // if this weren't a member function it would have to be defined as an auto as the original signature wouldn't work in cling
        // this is a known issue; see https://github.com/jupyter-xeus/xeus-cling/issues/40

        if (p->ch)
        {
            std::string ch(1, p->ch);
            return std::vector<std::pair<std::string, std::string>> {{ch, ""}};
        }
        else 
        {
            std::vector<std::pair<std::string, std::string>> from_left = getCodeCharPairs(p->left);
            std::vector<std::pair<std::string, std::string>> from_right = getCodeCharPairs(p->right);

            for (auto& pair : from_left)
                pair.second = '0' + pair.second;

            for (auto& pair : from_right)
                pair.second = '1' + pair.second;

            from_left.insert(from_left.end(), from_right.begin(), from_right.end());
            return from_left;
        }
    }

public:
    HuffmanCoding(const std::string& original_message) : original_message(original_message) {}
    
    std::string compress(const std::string& message){
        if (!huffmanTree)
            huffmanTree = buildHuffmanTree();
        if (codeCharPairs.size()==0)
            codeCharPairs = getCodeCharPairs(huffmanTree);

        std::unordered_map<std::string, std::string> codeTable(codeCharPairs.begin(), codeCharPairs.end());
        
        std::string compressed;
        for (const char& c : message)
        {
            std::string ch(1, c);
            compressed += codeTable[ch];
        }
        return compressed;
    }
    std::string decompress(const std::string& compressed){
        if (!huffmanTree)
            huffmanTree = buildHuffmanTree();
        if (codeCharPairs.size()==0)
            codeCharPairs = getCodeCharPairs(huffmanTree);

        for (auto& pair : codeCharPairs)
            std::swap(pair.first, pair.second);

        std::unordered_map<std::string, std::string> charTable(codeCharPairs.begin(), codeCharPairs.end());

        std::string::const_iterator it = compressed.begin();
        std::string code, decompressed;

        while (it!=compressed.end())
        {
            code += *it;
            if (charTable.find(code) != charTable.end()){
                decompressed += charTable[code];
                code = "";
            }
            it++;
        }
        return decompressed;
    }
};

std::string original_message(R"(In computer science and information theory, a Huffman code is a particular type of optimal prefix code that is commonly used for lossless data compression. The process of finding or using such a code proceeds by means of Huffman coding, an algorithm developed by David A. Huffman while he was a Sc.D. student at MIT, and published in the 1952 paper "A Method for the Construction of Minimum-Redundancy Codes".)");

original_message

"In computer science and information theory, a Huffman code is a particular type of optimal prefix code that is commonly used for lossless data compression. The process of finding or using such a code proceeds by means of Huffman coding, an algorithm developed by David A. Huffman while he was a Sc.D. student at MIT, and published in the 1952 paper "A Method for the Construction of Minimum-Redundancy Codes"."

In [8]:
HuffmanCoding hc(original_message);

original_message

"In computer science and information theory, a Huffman code is a particular type of optimal prefix code that is commonly used for lossless data compression. The process of finding or using such a code proceeds by means of Huffman coding, an algorithm developed by David A. Huffman while he was a Sc.D. student at MIT, and published in the 1952 paper "A Method for the Construction of Minimum-Redundancy Codes"."

In [9]:
std::string compressed = hc.compress(original_message);

compressed

""

In [3]:
std::string message = hc.decompress(compressed);

(

 
message

"ddddddddddddddddddd"

In [4]:


original_message

"ddddddddddddddddddd"

"In computer science and information theory, a Huffman code is a particular type of optimal prefix code that is commonly used for lossless data compression. The process of finding or using such a code proceeds by means of Huffman coding, an algorithm developed by David A. Huffman while he was a Sc.D. student at MIT, and published in the 1952 paper "A Method for the Construction of Minimum-Redundancy Codes"."

In [5]:
(decompressed == message)

true